In [3]:
import gc
import glob

import geopandas as gpd
import momepy as mm
import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [4]:
regions_buildings_dir = '/data/uscuni-ulce/regions/buildings/'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
overture_streets_dir = '/data/uscuni-ulce/overture_streets/'
streets_dir = '/data/uscuni-ulce/processed_data/streets/'
enclosures_dir = '/data/uscuni-ulce/processed_data/enclosures/'
tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'

regions_datadir = "/data/uscuni-ulce/"
eubucco_files = glob.glob(regions_datadir + "eubucco_raw/*")

In [3]:
# prague - 69333
# freiburg - 5883
# 4 - largest region

# df = pd.read_parquet('/data/uscuni-ulce/regions/cadastre_id_to_region.parquet')
# df.region.value_counts()
# '''
# region
# 4.0         11084489
# 107131.0     3078558
# 16242.0      1457482
# 4214.0       1389195
# 8707.0       1220411
#               ...   
# 7727.0         12682
# 6741.0         12235
# 10970.0        12170
# 10847.0        10871
# 15545.0        10409
# Name: count, Length: 828, dtype: int64
# '''

In [6]:
region_name = 69333
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_id, region_hull = region_hulls.loc[region_name].name, region_hulls.loc[region_name].convex_hull

## Buildings


In [5]:
from core.generate_buildings import read_region_buildings, process_region_buildings

In [6]:
# ## need to link eubucco building polygons to regions, this will change in the future
# building_region_mapping = pd.read_parquet(
#     regions_datadir + "regions/" + "id_to_region.parquet", engine="pyarrow"
# )
# typed_dict = pd.Series(
#     np.arange(building_region_mapping["id"].values.shape[0]),
#     index=building_region_mapping["id"].values,
# )
# region_ids = building_region_mapping.groupby("region")["id"].unique()
# del building_region_mapping  # its 2/3 gb

In [7]:
%%time

buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')

CPU times: user 228 ms, sys: 159 ms, total: 387 ms
Wall time: 367 ms


In [8]:
%%time

buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)

Percent polygons:  1.0
Final polygons:  457127 , dropped:  0.02918639964321368
CPU times: user 1min 15s, sys: 335 ms, total: 1min 15s
Wall time: 1min 15s


In [ ]:
buildings.to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

## Streets

In [9]:
from core.generate_streets import process_region_streets

In [ ]:
## overture is indexed based on 4326
overture_hull = region_hulls.loc[[region_name], ].to_crs(epsg=4326).convex_hull.iloc[0]

In [ ]:
%%time
## processs streets
streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)

/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:227: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:227: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:343: UserWarning: An error occured at location POINT (4634734.833449237 3007874.1868347814). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  roads_cleaned = simplify_singletons(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:227: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:200: UserWarning: Could not create a connectio

In [13]:
## save streets
streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

In [8]:
import lonboard
from sidecar import Sidecar

streets.geometry = streets.make_valid()
layer = lonboard.PathLayer.from_geopandas(streets.to_crs(4326), width_min_pixels=1)
sc = Sidecar(title='streets')
m = lonboard.Map(layer)
with sc:
    display(m)

In [ ]:
# streets.explore(tiles='cartodbpositron', prefer_canvas=True)

In [ ]:
1

In [50]:
buildings.shape

(8756586, 5)

In [ ]:
import geopandas as gpd
# gpd.read_parquet(streets_dir + f'streets_{region_id}.parquet').explore()

In [13]:
# gpd.read_parquet(enclosures_dir + f'enclosure_{region_id}.parquet').explore()

## Enclosures & Tessellation

In [14]:
from core.generate_elements import process_region_elements

In [15]:
%%capture cap
enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)

In [16]:
enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
print("Processed enclosures")

## save files
tesselations.to_parquet(
    tessellations_dir + f"tessellation_{region_id}.parquet"
)
print("processed tesselations")

Processed enclosures
processed tesselations


In [17]:
cap.show()

---- Processing region:  69333 2024-10-01 12:40:26.099743


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
# import lonboard
# layer = lonboard.PolygonLayer.from_geopandas(tesselations, opacity=0.15)
# m = lonboard.Map([layer])
# m

In [ ]:
# layer = lonboard.PolygonLayer.from_geopandas(enclosures, opacity=0.15)
# m = lonboard.Map([layer])
# m

## Graphs

In [18]:
from core.generate_ngraphs import process_region_graphs

In [19]:
%%time
process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
)

Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 243 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
CPU times: user 41.9 s, sys: 958 ms, total: 42.8 s
Wall time: 42.7 s


## Characters

In [20]:
from core.generate_chars import process_single_region_chars, process_street_chars

In [21]:
%%capture cap

process_single_region_chars(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir
)

## Generate primary data

In [22]:
tessellation = gpd.read_parquet(chars_dir + f"tessellations_chars_{region_id}.parquet")
buildings = gpd.read_parquet(chars_dir + f"buildings_chars_{region_id}.parquet")
enclosures = gpd.read_parquet(chars_dir + f"enclosures_chars_{region_id}.parquet")
streets = gpd.read_parquet(chars_dir + f"streets_chars_{region_id}.parquet")
nodes = gpd.read_parquet(chars_dir + f"nodes_chars_{region_id}.parquet")

In [23]:

merged = pd.merge(
    tessellation.drop(columns=["geometry"]),
    buildings.drop(columns=["nodeID", "geometry", 'nID']),
    right_index=True,
    left_index=True,
    how="left",
)

merged = merged.merge(
    enclosures.drop(columns="geometry"),
    right_on="eID",
    left_on="enclosure_index",
    how="left",
)

merged = merged.merge(streets.drop(columns="geometry"), on="nID", how="left")
merged = merged.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")

merged = merged.drop(
    columns=[
        "nID",
        "eID",
        "nodeID",
        "mm_len",
        "cdsbool",
        "node_start",
        "node_end",
        "x",
        "y",
        "enclosure_index",
        # "id",
        # "osm_id",
        "index",  ## maybe keep
    ]
)
merged = merged.set_index(tessellation.index)

In [24]:
from core.utils import used_keys

In [25]:
primary = merged[list(used_keys.keys())]
primary.shape

(461350, 63)

In [26]:
primary.to_parquet(chars_dir + f'primary_chars_{region_id}.parquet')